In [ ]:
!pip install gradio tensorflow opencv-python


In [ ]:
import tensorflow as tf
import numpy as np

# Load dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()  # just a placeholder

print(X_train.shape, y_train.shape)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(60000, 28, 28) (60000,)


In [ ]:
!wget -q https://github.com/ardamavi/Sign-Language-Digits-Dataset/archive/master.zip
!unzip -q master.zip


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

IMG_SIZE = 64
data = []
labels = []

path = "/content/Sign-Language-Digits-Dataset-master"

for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    if os.path.isdir(folder_path):  # Add this check to process only directories
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # grayscale
            if img is not None: # Add this check
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                data.append(img)
                labels.append(int(folder))  # folder name = class label
            else:
                print(f"Warning: Could not load image {img_path}") # Optional: print warning


data = np.array(data).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

ValueError: invalid literal for int() with base 10: 'Examples'

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

IMG_SIZE = 64
data = []
labels = []

path = "Sign-Language-Digits-Dataset-master/Dataset"

for folder in os.listdir(path):
    if not folder.isdigit():  # ✅ Skip non-numeric folders like "Examples"
        continue

    folder_path = os.path.join(path, folder)
    for img_file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"⚠️ Skipping invalid image: {img_path}")
            continue
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        data.append(img)
        labels.append(int(folder))  # class = folder number

data = np.array(data).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

print("✅ Data loaded successfully")
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)


✅ Data loaded successfully
Train shape: (1649, 64, 64, 1) Test shape: (413, 64, 64, 1)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 classes (digits 0–9)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,914 (2.61 MB)

 Trainable params: 683,914 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=5,
    validation_data=(X_test, y_test),
    batch_size=32
)


Epoch 1/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 11s 209ms/step - accuracy: 0.5884 - loss: 1.1755 - val_accuracy: 0.8136 - val_loss: 0.6694
Epoch 2/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 181ms/step - accuracy: 0.7323 - loss: 0.7811 - val_accuracy: 0.8668 - val_loss: 0.4986
Epoch 3/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 180ms/step - accuracy: 0.8069 - loss: 0.6037 - val_accuracy: 0.8717 - val_loss: 0.4268
Epoch 4/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 11s 192ms/step - accuracy: 0.8314 - loss: 0.5004 - val_accuracy: 0.9104 - val_loss: 0.3483
Epoch 5/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 11s 212ms/step - accuracy: 0.8620 - loss: 0.4022 - val_accuracy: 0.9153 - val_loss: 0.3321


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {test_acc*100:.2f}%")


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.9011 - loss: 0.4625
✅ Test Accuracy: 91.53%


In [ ]:
model.save("hand_gesture_model.h5")
print("Model saved as hand_gesture_model.h5")


Model saved as hand_gesture_model.h5


In [ ]:
import gradio as gr
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load model
model = load_model("hand_gesture_model.h5")

IMG_SIZE = 64
class_names = [str(i) for i in range(10)]  # 0–9 gestures

def predict_gesture(image):
    # Convert to grayscale
    img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Resize
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
    # Expand dims for model input
    img = np.expand_dims(img, axis=(0, -1))

    preds = model.predict(img)
    gesture = class_names[np.argmax(preds)]
    return f"Predicted Gesture: {gesture}"

# Create Gradio app
demo = gr.Interface(
    fn=predict_gesture,
    inputs=gr.Image(type="numpy", label="Upload Hand Gesture"),
    outputs="text",
    title="✋ Hand Gesture Recognition",
    description="Upload a hand gesture image (0–9 in sign language)."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://599f5919c348ca232c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
